In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import re
import numpy as np
import pandas as pd
import pickle
from torch.optim.lr_scheduler import StepLR

In [2]:
#Hyperparameter

hidden_size = 256
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
UNK_TOKEN = 3
MAX_LENGTH = 300
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
def clean_text(text):
    if pd.isna(text): # NaN 값 처리
        return ''
    text = text.lower()
    text = re.sub(r'\d+', ' ', text) # 숫자는 공백으로 처리
    text = re.sub(r'([^\w\s])', r' \1 ', text) # 마침표 앞 뒤로 공백 추가
    text = re.sub(r'\s+', ' ', text) # 두 개 이상의 공백을 하나로 처리
    text = text.strip() # 텍스트 양 옆의 공백 제거
    
    return text

In [5]:
from mecab import MeCab
mecab = MeCab()

In [6]:
def indiceFromSentence(vocab, sentence):
    return [vocab.get(word, vocab['<UNK>']) for word in mecab.morphs(sentence)]

In [7]:
def tensorFromSentence(vocab, sentence):
    indice = indiceFromSentence(vocab, sentence)
    indice.append(EOS_TOKEN)
    return torch.tensor(indice, dtype=torch.long, device=device).view(-1, 1)

In [8]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden
    
    def initHidden(self):
        return(torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))

In [9]:
class AttentionDecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(AttentionDecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, self.hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden, encoder_outputs):
        
        embedded = self.embedding(input).view(1, 1, -1)
        attention_weights = F.softmax(torch.bmm(encoder_outputs.unsqueeze(0), hidden[0][0].unsqueeze(2)).squeeze(2), dim=1) # batch matrix multiplication
        attention_applied = torch.bmm(attention_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        
        new_hidden = (torch.vstack([attention_applied, attention_applied]), hidden[1])
        output, hidden = self.lstm(embedded[0].unsqueeze(0), new_hidden)
        output = self.out(output[0])
        return output, hidden, attention_weights
    
    def initHidden(self):
        return(torch.zeros(2, 1, self.hidden_size, device=device), torch.zeros(2, 1, self.hidden_size, device=device))

In [10]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    
    loss = 0    
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] += encoder_output[0, 0]
        
    decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
    decoder_hidden = encoder_hidden
    
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        loss += criterion(decoder_output, target_tensor[di])
        
        if decoder_input.item() == EOS_TOKEN:
            break
        
    loss.backward() # 역전파 
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length
    
    

In [11]:
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01):
    print_loss_total = 0
    
    scheduler_encoder = StepLR(encoder_optimizer, step_size=10, gamma=0.1) # 10번마다 학습률이 10%로 줄인다.
    scheduler_decoder = StepLR(decoder_optimizer, step_size=10, gamma=0.1)
    min_loss = 1000000
    
    for iter in range(1, n_iters+1):
        training_pair = random.choice(pairs) # input - target pair
        input_tensor = tensorFromSentence(word_to_idx, training_pair[0]).to(device)
        target_tensor = tensorFromSentence(word_to_idx, training_pair[1]).to(device)
        
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print(f'Iteration : {iter}, Loss : {print_loss_avg: .4f}')
            print_loss_total = 0
            
            if min_loss > print_loss_avg:
                torch.save(encoder.state_dict(), './encoder_seq2seq_attention_dot_' + str(iter) + '.pth')
                torch.save(decoder.state_dict(), '/decoder_seq2seq_attention_dot_' + str(iter) + '.pth')
                min_loss = print_loss_avg
            
            scheduler_encoder.step()
            scheduler_decoder.step()

In [12]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(word_to_idx, sentence).to(device)
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
            
        decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = [] # output sentence
        
        decoder_attentions = torch.zeros(max_length, max_length)
        
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_TOKEN:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(idx_to_word[topi.item()]) # 최종 아웃풋의 index
            
            decoder_input = topi.squeeze().detach()
        
        meaningful_words = [word for word in decoded_words if word not in ('<EOS>', '<UNK>')]
        final_output = ' '.join(meaningful_words)
        return final_output

In [13]:
def chat(encoder, decoder, max_length=MAX_LENGTH):
    print("Let's chat (type 'bye' to exit)")
    while True:
        input_sentence = input(">>")
        if input_sentence == 'bye':
            break
        output_sentence = evaluate(encoder, decoder, input_sentence)
        print('<', output_sentence)

In [14]:
# load data and preprocessing
df = pd.read_csv('./dementia_fix.csv', sep=',', names=['Question', 'Intention', 'Answer'], skiprows=1)
df['Encoder Inputs'] = df['Question'].apply(clean_text)
df['Decoder Inputs'] = df['Answer'].apply(clean_text)


In [15]:
df['Encoder Inputs']

0       알츠하이머병의 원인으로 매일 소주를 섭취하는 것이 언급되고 있는데 , 이에 대한 근...
1                         알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요 ?
2                     알츠하이머병의 발생 원인에 대한 연구나 발견이 진행 중인가요 ?
3              알츠하이머병의 발병과 관련하여 뇌의 노화로 인한 증상과 원인을 알려주세요 .
4                   알츠하이머병의 원인과 관련된 연구 결과가 있을까요 ? 알려주세요 .
                              ...                        
6618                         치매 치료에는 어떤 운동이나 작업이 효과적일까요 ?
6619    치매 치료의 결과와 과정을 상세히 설명해주세요 . 치매 치료의 효과는 어떻게 나타날...
6620                      치매를 치료하기 위해 어떤 치료 방법들이 효과적일까요 ?
6621                        치매 치료를 위해 어떤 약물이 사용될 수 있을까요 ?
6622                         치매 치료를 위해 어떤 전문가와 협력해야 할까요 ?
Name: Encoder Inputs, Length: 6623, dtype: object

In [16]:
df['Decoder Inputs']

0       알츠하이머병의 정확한 원인은 아직 밝혀지지 않았지만 , 연구들이 알츠하이머병의 발병...
1       알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다 . 알츠하이머병은 아직 완...
2       알츠하이머병은 치매를 일으키는 가장 흔한 퇴행성 뇌질환으로 , 년 독일 의사 알로이...
3       알츠하이머병은 현재까지 그 발병 원인에 대한 완벽한 해명은 아직 이루어지지 않았습니...
4       알츠하이머병은 복잡한 질환으로 , 아직도 원인이 완전히 밝혀진 것은 아닙니다 . 그...
                              ...                        
6618    치매는 노인들에게 주로 발생하는 뇌질환으로 , 원인과 치료 방법은 아직 완전히 밝혀...
6619    치매는 일상 생활을 수행하는 능력을 심각하게 손상시키는 질환으로 , 후천성 치매와 ...
6620    치매는 노화로 인해 기억력과 지능을 점차적으로 잃는 질병으로 , 알츠하이머병이 주요...
6621    알츠하이머병은 뇌에 변화가 생겨서 인지 기능에 장애가 생기는 신경퇴행성 질환입니다 ...
6622    치매는 현재까지 완전한 치료가 불가능한 치매입니다 . 치매는 다양한 원인에 의해 발...
Name: Decoder Inputs, Length: 6623, dtype: object

In [17]:
input_sentence = [sentence for sentence in df['Encoder Inputs']]
output_sentence = [sentence + "<EOS>" for sentence in df['Decoder Inputs']]

In [18]:
input_sentence[0:5]

['알츠하이머병의 원인으로 매일 소주를 섭취하는 것이 언급되고 있는데 , 이에 대한 근거가 있는지 알려주세요 .',
 '알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요 ?',
 '알츠하이머병의 발생 원인에 대한 연구나 발견이 진행 중인가요 ?',
 '알츠하이머병의 발병과 관련하여 뇌의 노화로 인한 증상과 원인을 알려주세요 .',
 '알츠하이머병의 원인과 관련된 연구 결과가 있을까요 ? 알려주세요 .']

In [19]:
output_sentence[0:5]

['알츠하이머병의 정확한 원인은 아직 밝혀지지 않았지만 , 연구들이 알츠하이머병의 발병 기전에 대해 논의하고 있습니다 . 일부 연구에 따르면 , 유전적인 요소와 뇌의 기능 손상이 관련되어 있다고 알려져 있습니다 . 알츠하이머병은 아밀로이드 베타 단백질과 타우 단백질의 과도한 생성 , 뇌 세포의 비정상적인 활동 , 뇌 조직의 변화로 인해 발생하는 것으로 생각되고 있습니다 . 이러한 변화가 알츠하이머병의 발병 위험을 증가시키고 , 병의 진행을 가속화시킨다는 것입니다 . 알츠하이머병의 발병과 관련된 위험 요소에 대해서는 더 많은 연구와 조사가 필요합니다 . 더 많은 연구와 자료 수집을 통해 알츠하이머병에 대한 더 많은 이해와 예방 방법이 개발될 것으로 기대됩니다 .<EOS>',
 '알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다 . 알츠하이머병은 아직 완전히 이해되지 않았지만 , 연구 결과에 따르면 유전적인 요소와 다양한 환경적인 요인이 이 질환을 일으키는 역할을 한다고 알려져 있습니다 . 특히 , 아밀로이드 베타 단백질의 비정상적인 축적이 알츠하이머병과 관련이 있는 것으로 알려져 있습니다 . 이 외에도 나이 , 노화 , 고혈압 , 당뇨병 , 그리고 흡연 등과 같은 다른 요인들도 알츠하이머병 발병과 연관성이 있을 수 있습니다 . 더 많은 연구와 조사를 통해 알츠하이머병의 원인을 파악하고 예방 방법을 개발할 필요가 있습니다 .<EOS>',
 '알츠하이머병은 치매를 일으키는 가장 흔한 퇴행성 뇌질환으로 , 년 독일 의사 알로이스 알츠하이머에 의해 처음으로 보고되었습니다 . 이 질환의 원인에 대해서는 현재까지 명확한 답은 없으나 , 치매 발생의 위험 요소와 관련하여 몇 가지 위험 요인이 알려져 있습니다 . 일반적으로 , 가장 잘 알려진 요인 중 하나는 고령입니다 . 고령은 치매의 발병 위험을 증가시키는 가장 큰 위험 요소로 알려져 있습니다 . 또한 , 가족력이 있는 경우 알츠하이머병 발생 위험이 높아집니다 . 연구에 따르면 , 조발성 가족성 알츠하이머병은 주로 

In [55]:
# 단어 사전 생성
all_word = set(' '.join(df['Encoder Inputs'].tolist() + df['Decoder Inputs'].tolist()).split())
vocab = {'<PAD>': PAD_TOKEN, '<SOS>': SOS_TOKEN, '<EOS>': EOS_TOKEN, '<UNK>': UNK_TOKEN}
vocab.update({word: i+4 for i, word in enumerate(all_word)})
vocab_size = len(vocab)

with open('vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)

In [20]:
with open('word_to_idx.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [21]:
vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 '알츠하이머병': 4,
 '의': 5,
 '원인': 6,
 '으로': 7,
 '매일': 8,
 '소주': 9,
 '를': 10,
 '섭취': 11,
 '하': 12,
 '는': 13,
 '것': 14,
 '이': 15,
 '언급': 16,
 '되': 17,
 '고': 18,
 '있': 19,
 '는데': 20,
 ',': 21,
 '에': 22,
 '대한': 23,
 '근거': 24,
 '가': 25,
 '는지': 26,
 '알려': 27,
 '주': 28,
 '세요': 29,
 '.': 30,
 '라는': 31,
 '질병': 32,
 '은': 33,
 '유전': 34,
 '적': 35,
 '영향': 36,
 '을': 37,
 '받': 38,
 '인가요': 39,
 '?': 40,
 '발생': 41,
 '연구': 42,
 '나': 43,
 '발견': 44,
 '진행': 45,
 '중': 46,
 '발병': 47,
 '과': 48,
 '관련': 49,
 '여': 50,
 '뇌': 51,
 '노화': 52,
 '로': 53,
 '인한': 54,
 '증상': 55,
 '된': 56,
 '결과': 57,
 '을까요': 58,
 '무엇': 59,
 '주요': 60,
 '예방': 61,
 '위해': 62,
 '한': 63,
 '병': 64,
 '씩': 65,
 '권장': 66,
 '이유': 67,
 '다른': 68,
 '들': 69,
 '대해': 70,
 '데': 71,
 '미치': 72,
 '요인': 73,
 '어떤': 74,
 '나요': 75,
 '치매': 76,
 '사이': 77,
 '어떠': 78,
 '연관': 79,
 '성': 80,
 '하나': 81,
 '인': 82,
 '대해서': 83,
 '자세히': 84,
 '가족력': 85,
 '이외': 86,
 '상세': 87,
 '게': 88,
 '설명': 89,
 '해': 90,
 '그': 91,
 '따른': 92,
 '

In [56]:
all_word

{'실행되며',
 '줄여주고',
 '대비를',
 '물을',
 '보호제를',
 '아포지단백질',
 '빨라지면',
 '직장에서의',
 '관리하는지',
 '감소의',
 '이름으로',
 '다운증후군',
 '개선시켜',
 '시도와',
 '인지를',
 '강화하며',
 '집중하지',
 '느릴',
 '완치가',
 '플루니트라제팜',
 '보행장애',
 '청소년이나',
 '시작되면',
 '질문지를',
 '식단이',
 '주어진',
 '감퇴는',
 '체크업을',
 '움직임을',
 '차와',
 '상태임에도',
 '결핍을',
 '지원은',
 '요일이나',
 '사례를',
 '길항제는',
 '권장됩니다',
 '평상',
 '임상심리검사가',
 '이상들이',
 'c가',
 '유지하시기',
 '남용하면',
 '집중력의',
 '전두측두엽과',
 '전기경련치료',
 '저해합니다',
 '전기요법',
 '발견되었을',
 '맞춤화해야',
 '방지할',
 'vmadan',
 '떠오를',
 '빈도와',
 '말이나',
 '종양',
 '해소하기',
 '진단과정에서',
 '발생하는데에',
 '필요하다는',
 '이르기까지',
 '손상시키면',
 '부신기능부전',
 '때와',
 '감소시킴으로써',
 '전달에',
 '정신건강이',
 '징후입니다',
 '범위에',
 '의존하거나',
 '동안',
 '안전하며',
 '기억상실',
 '압력으로',
 '부적합한',
 '모델에',
 '이전과는',
 '이해하는',
 '깨지는',
 '분야이며',
 '지속하는',
 '혈류순환을',
 '보존하는',
 '신경질환의',
 '구조물입니다',
 '치료법을',
 '병용해서',
 '의료기관을',
 '뇌기능활성도검사도',
 '싸움에서',
 '존재하지',
 '공급하는데',
 '무기력증이',
 '주요우울증의',
 '연구하는',
 '호소합니다',
 '통합한',
 '공급되어야',
 '하지는',
 '도구와',
 '노인에게는',
 '바람직합니다',
 '끊었을',
 '내용까지',
 '가져오며',
 '어떤지를',
 '말할',
 '많기',
 '인격장애나',
 '불

In [57]:
vocab

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 '실행되며': 4,
 '줄여주고': 5,
 '대비를': 6,
 '물을': 7,
 '보호제를': 8,
 '아포지단백질': 9,
 '빨라지면': 10,
 '직장에서의': 11,
 '관리하는지': 12,
 '감소의': 13,
 '이름으로': 14,
 '다운증후군': 15,
 '개선시켜': 16,
 '시도와': 17,
 '인지를': 18,
 '강화하며': 19,
 '집중하지': 20,
 '느릴': 21,
 '완치가': 22,
 '플루니트라제팜': 23,
 '보행장애': 24,
 '청소년이나': 25,
 '시작되면': 26,
 '질문지를': 27,
 '식단이': 28,
 '주어진': 29,
 '감퇴는': 30,
 '체크업을': 31,
 '움직임을': 32,
 '차와': 33,
 '상태임에도': 34,
 '결핍을': 35,
 '지원은': 36,
 '요일이나': 37,
 '사례를': 38,
 '길항제는': 39,
 '권장됩니다': 40,
 '평상': 41,
 '임상심리검사가': 42,
 '이상들이': 43,
 'c가': 44,
 '유지하시기': 45,
 '남용하면': 46,
 '집중력의': 47,
 '전두측두엽과': 48,
 '전기경련치료': 49,
 '저해합니다': 50,
 '전기요법': 51,
 '발견되었을': 52,
 '맞춤화해야': 53,
 '방지할': 54,
 'vmadan': 55,
 '떠오를': 56,
 '빈도와': 57,
 '말이나': 58,
 '종양': 59,
 '해소하기': 60,
 '진단과정에서': 61,
 '발생하는데에': 62,
 '필요하다는': 63,
 '이르기까지': 64,
 '손상시키면': 65,
 '부신기능부전': 66,
 '때와': 67,
 '감소시킴으로써': 68,
 '전달에': 69,
 '정신건강이': 70,
 '징후입니다': 71,
 '범위에': 72,
 '의존하거나': 73,
 '동안': 74,
 '안전하며': 75,
 '기억상실': 76,

In [22]:
word_to_idx = vocab
idx_to_word = {i: word for word, i in word_to_idx.items()}

In [23]:
word_to_idx

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<UNK>': 3,
 '알츠하이머병': 4,
 '의': 5,
 '원인': 6,
 '으로': 7,
 '매일': 8,
 '소주': 9,
 '를': 10,
 '섭취': 11,
 '하': 12,
 '는': 13,
 '것': 14,
 '이': 15,
 '언급': 16,
 '되': 17,
 '고': 18,
 '있': 19,
 '는데': 20,
 ',': 21,
 '에': 22,
 '대한': 23,
 '근거': 24,
 '가': 25,
 '는지': 26,
 '알려': 27,
 '주': 28,
 '세요': 29,
 '.': 30,
 '라는': 31,
 '질병': 32,
 '은': 33,
 '유전': 34,
 '적': 35,
 '영향': 36,
 '을': 37,
 '받': 38,
 '인가요': 39,
 '?': 40,
 '발생': 41,
 '연구': 42,
 '나': 43,
 '발견': 44,
 '진행': 45,
 '중': 46,
 '발병': 47,
 '과': 48,
 '관련': 49,
 '여': 50,
 '뇌': 51,
 '노화': 52,
 '로': 53,
 '인한': 54,
 '증상': 55,
 '된': 56,
 '결과': 57,
 '을까요': 58,
 '무엇': 59,
 '주요': 60,
 '예방': 61,
 '위해': 62,
 '한': 63,
 '병': 64,
 '씩': 65,
 '권장': 66,
 '이유': 67,
 '다른': 68,
 '들': 69,
 '대해': 70,
 '데': 71,
 '미치': 72,
 '요인': 73,
 '어떤': 74,
 '나요': 75,
 '치매': 76,
 '사이': 77,
 '어떠': 78,
 '연관': 79,
 '성': 80,
 '하나': 81,
 '인': 82,
 '대해서': 83,
 '자세히': 84,
 '가족력': 85,
 '이외': 86,
 '상세': 87,
 '게': 88,
 '설명': 89,
 '해': 90,
 '그': 91,
 '따른': 92,
 '

In [24]:
idx_to_word

{0: '<PAD>',
 1: '<SOS>',
 2: '<EOS>',
 3: '<UNK>',
 4: '알츠하이머병',
 5: '의',
 6: '원인',
 7: '으로',
 8: '매일',
 9: '소주',
 10: '를',
 11: '섭취',
 12: '하',
 13: '는',
 14: '것',
 15: '이',
 16: '언급',
 17: '되',
 18: '고',
 19: '있',
 20: '는데',
 21: ',',
 22: '에',
 23: '대한',
 24: '근거',
 25: '가',
 26: '는지',
 27: '알려',
 28: '주',
 29: '세요',
 30: '.',
 31: '라는',
 32: '질병',
 33: '은',
 34: '유전',
 35: '적',
 36: '영향',
 37: '을',
 38: '받',
 39: '인가요',
 40: '?',
 41: '발생',
 42: '연구',
 43: '나',
 44: '발견',
 45: '진행',
 46: '중',
 47: '발병',
 48: '과',
 49: '관련',
 50: '여',
 51: '뇌',
 52: '노화',
 53: '로',
 54: '인한',
 55: '증상',
 56: '된',
 57: '결과',
 58: '을까요',
 59: '무엇',
 60: '주요',
 61: '예방',
 62: '위해',
 63: '한',
 64: '병',
 65: '씩',
 66: '권장',
 67: '이유',
 68: '다른',
 69: '들',
 70: '대해',
 71: '데',
 72: '미치',
 73: '요인',
 74: '어떤',
 75: '나요',
 76: '치매',
 77: '사이',
 78: '어떠',
 79: '연관',
 80: '성',
 81: '하나',
 82: '인',
 83: '대해서',
 84: '자세히',
 85: '가족력',
 86: '이외',
 87: '상세',
 88: '게',
 89: '설명',
 90: '해',
 91: '그',
 92: '따른',
 9

In [26]:
word_to_idx['지연']

998

In [27]:
idx_to_word[998]

'지연'

In [29]:
vocab_size = len(vocab)
vocab_size

5030

In [30]:
encoder = EncoderLSTM(vocab_size, hidden_size).to(device)
decoder = AttentionDecoderLSTM(hidden_size, vocab_size).to(device)

In [31]:
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.005)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

In [32]:
pairs = [list(x) for x in zip(df['Encoder Inputs'], df['Decoder Inputs'])]

In [33]:
pairs[1]

['알츠하이머병이라는 질병은 유전적 영향을 받는 것인가요 ?',
 '알츠하이머병은 현재까지 완전한 원인이 밝혀지지 않았습니다 . 알츠하이머병은 아직 완전히 이해되지 않았지만 , 연구 결과에 따르면 유전적인 요소와 다양한 환경적인 요인이 이 질환을 일으키는 역할을 한다고 알려져 있습니다 . 특히 , 아밀로이드 베타 단백질의 비정상적인 축적이 알츠하이머병과 관련이 있는 것으로 알려져 있습니다 . 이 외에도 나이 , 노화 , 고혈압 , 당뇨병 , 그리고 흡연 등과 같은 다른 요인들도 알츠하이머병 발병과 연관성이 있을 수 있습니다 . 더 많은 연구와 조사를 통해 알츠하이머병의 원인을 파악하고 예방 방법을 개발할 필요가 있습니다 .']

In [34]:
trainIters(encoder, decoder, 1000, 100)

Iteration : 100, Loss :  5.7404
Iteration : 200, Loss :  5.4517
Iteration : 300, Loss :  5.3589
Iteration : 400, Loss :  5.4354
Iteration : 500, Loss :  5.4003
Iteration : 600, Loss :  5.4396
Iteration : 700, Loss :  5.3403
Iteration : 800, Loss :  5.3764
Iteration : 900, Loss :  5.3919
Iteration : 1000, Loss :  5.3892


In [35]:
encoder.eval()
decoder.eval()

AttentionDecoderLSTM(
  (embedding): Embedding(5030, 256)
  (lstm): LSTM(256, 256, num_layers=2)
  (out): Linear(in_features=256, out_features=5030, bias=True)
)

In [36]:
chat(encoder, decoder)

Let's chat (type 'bye' to exit)
< 알코올 은 치매 는 는 는 는 는 는 는 는 는 는 는 는 . 는 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
< 알코올 은 치매 는 는 는 는 는 는 는 는 . 는 . 는 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 